In [ ]:
#Required Libraries
import pandas as pd
from bs4 import BeautifulSoup
import warnings
import re
import os
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import openai
import time
warnings.filterwarnings("ignore")
openai.api_key = '' # use your open ai api key
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize

In [ ]:
#Below functions were used to extract the data from Aquaint, TimeBank and TempEval. Comment and uncomment the lines below based on the structure.
#creating event and relation dataframe
def create_event_rel_pd(root_node):
    temp_mi=[]
    temp_tlink=[]
    for child in root_node:
        if child.tag == 'MAKEINSTANCE':
            temp_mi.append({'eventID':child.attrib['eventID'],'eiid':child.attrib['eiid'],'tense':child.attrib['tense'],'aspect':child.attrib['aspect'],
            'polarity':child.attrib['polarity'],'pos':child.attrib['pos'],'event':'','context':''})
        if child.tag == 'TLINK' and  'eventInstanceID' in str(child.attrib) and 'relatedToEventInstance' in str(child.attrib):
            temp_tlink.append({'lid':child.attrib['lid'],'relType':child.attrib['relType'],'eventInstanceID':child.attrib['eventInstanceID'],'relatedToEventInstance':child.attrib['relatedToEventInstance']})
    event_pd=pd.DataFrame.from_dict(temp_mi)
    rel_pd=pd.DataFrame.from_dict(temp_tlink)
    return event_pd,rel_pd

# mapping the events to its event id
def populate_event(root_node,event_pd):
    # # for txt in root_node.findall('TEXT'):
    #     # for evt in txt.findall('EVENT'):
    # # for evt in root_node.findall('DOC')[0].findall('BODY')[0].findall('TEXT')[0]:
    # for evt in root_node.findall('EVENT'):
    #     # for events in evt.findall('EVENT'):
    #     for events in evt.attrib:
    #         if evt.attrib.get("eid") is not None:
    #     # root_node.findall('DOC')[0].findall('BODY')[0].findall('TEXT')[0].findall('EVENT'):
    #             idx=event_pd.index[event_pd['eventID']==evt.attrib['eid']].tolist()[0]
    #             if idx>-1:
    #                 event_pd.at[idx,'event'] = evt.text
    for events in root_node.findall('DOC')[0].findall('BODY')[0].findall('TEXT')[0].findall('P'):
        for evt in events.findall('EVENT'):
            if evt.attrib.get("eid") is not None:
                idx=event_pd.index[event_pd['eventID']==evt.attrib['eid']].tolist()[0]
                if idx>-1:
                    event_pd.at[idx,'event'] = evt.text
    return event_pd

# removing tags
def remove_tags(root_node):
    # for txt in root_node.findall('TEXT'):
    #     for events in txt.findall('EVENT'):
    # for p in  root_node.findall('DOC')[0].findall('BODY')[0].findall('TEXT'):
    #     for events in p.findall('EVENT'):
    for evt in root_node.findall('DOC')[0].findall('BODY')[0].findall('TEXT')[0].findall('P'):
        for events in evt.findall('EVENT'):
    # root_node.findall('DOC')[0].findall('BODY')[0].findall('TEXT')[0].findall('EVENT'):
            events.text = '('+events.attrib['eid'] + ':'+events.text+')'
    xmlstr = ET.tostring(root_node, encoding='utf8')
    cleantext = BeautifulSoup(xmlstr, "lxml").text
    cleantext=cleantext.replace('\n','')
    return cleantext

#merging all the dataframes created into a single dataframe having all the details
def merge_data(event_pd,rel_pd):
    rel_event_pd = pd.merge(rel_pd, event_pd,  how='left', left_on='eventInstanceID', right_on = 'eiid')
    rel_event_all_pd = pd.merge(rel_event_pd, event_pd,  how='left', left_on='relatedToEventInstance', right_on = 'eiid')
    relation_pd=rel_event_all_pd.rename(columns={'relType':'Relation','lid':'lid','eventInstanceID':'eiid1','relatedToEventInstance':'eiid2',
    'eventID_x':'eventid1','eventID_y':'eventid2','tense_x':'e1tense', 'event_x':'event1','tense_y':'e2tense','event_y':'event2','context_x':'context'})
    relation_pd=relation_pd.drop(['eiid_x','aspect_x','polarity_x','pos_x','eiid_y','aspect_y','polarity_y','pos_y','context_y'],axis=1)
    relation_pd = relation_pd.reset_index()
    return relation_pd

# Extracting the irght context for the given 2 events.
def prepare_context(relation_pd,text):
    results=[]
    for idx,row in relation_pd.iterrows():
        if row.eiid1 < row.eiid2:
            test1='('+row.eventid1+':'+row.event1+')'
            test2='('+row.eventid2+':'+row.event2+')'
        else:
            test1='('+row.eventid2+':'+row.event2+')'
            test2='('+row.eventid1+':'+row.event1+')'
        search_regex = r'([^.]*{0}[^.]*[^.]*{1}[^.]*)'.format(test1,test2)
        res=re.findall(search_regex, text)
        if res ==[]:
            res=re.findall( r'([^.]*.*{0}*{1}[^.]*)'.format(test1,test2), text)
        results.append(res[0][0])
    relation_pd['context']=results

# def get_completion(prompt, model='gpt-3.5-turbo'):
#   messages=[{"role":"user","content":prompt}]
#   response=openai.ChatCompletion.create(model=model,messages=messages,temperature=0)
#   return response.choices[0].message['content']

In [ ]:
path=r'..\Data\aquaint_timeml_1.0\aquaint_timeml_1.0\data\N35' # specify the path to your datafile
dir_list = os.listdir(path)
print(len(dir_list))

10


In [ ]:

for files in dir_list:
    root_node = ET.parse(path+'\\'+files).getroot()
    event_pd,rel_pd=create_event_rel_pd(root_node)
    event_pd=populate_event(root_node,event_pd)
    text=remove_tags(root_node)
    merged_data=merge_data(event_pd,rel_pd)
    total_len=total_len+len(merged_data)
    prepare_context(merged_data,text)
    print(merged_data.head())
    print("*********************************************************************")
    print("*********************************************************************")
    merged_data.to_csv('ExplicitQA.csv',mode='a', index=False, header=False)
    # ,mode='a', index=False, header=False


3
<Element 'TimeML' at 0x000002344A9A3310>
09:34:19
   index lid Relation eiid1 eiid2 eventid1 e1tense    event1  \
0      0  l7   BEFORE   ei5   ei6       e5    NONE  speeches   

                                             context eventid2 e2tense  \
0  In (e5:speeches) before the signing (e6:ceremo...       e6    NONE   

     event2  
0  ceremony  
*********************************************************************
*********************************************************************
<Element 'TimeML' at 0x000002344A9B5860>
09:34:19
   index lid Relation eiid1 eiid2 eventid1   e1tense      event1  \
0      0  l5    AFTER   ei3   ei4       e3  PRESPART  shattering   

                                             context eventid2 e2tense  \
0  0173  1999-03-13  Hungarian Leaders Hail Entry...       e4    NONE   

         event2  
0  World War II  
*********************************************************************
****************************************************************

Once the dataframe is ready, manually correct the context if any error found or alter it according to your needs.

In [ ]:
data=pd.read_csv('ExplicitQA.csv')
print(len(data))
data

1035


,Index,lid,Relation,eiid1,eiid2,eventid1,e1tense,event1,context,eventid2,e2tense,event2
0,0,l3,IBEFORE,ei1,ei2,e1,PAST,exploded,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e2,PRESPART,killing
1,1,l4,BEFORE,ei2,ei3,e2,PRESPART,killing,embassies in the Kenyan and Tanzanian capitals...,e3,PAST,said
2,2,l5,AFTER,ei3,ei12,e3,PAST,said,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e12,PAST,injured
3,3,l6,BEFORE,ei12,ei4,e12,PAST,injured,\t The American ambassador to Kenya was amon...,e4,PAST,said
4,4,l7,BEFORE,ei1,ei5,e1,PAST,exploded,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e5,PAST,said
...,...,...,...,...,...,...,...,...,...,...,...,...
1030,0,l30,AFTER,ei20,ei22,e20,PRESENT,finished,The three countries have (e20:finished) allne...,e22,PAST,ratified
1031,0,l7,BEFORE,ei5,ei6,e5,NONE,speeches,In (e5:speeches) before the signing (e6:ceremo...,e6,NONE,ceremony
1032,0,l5,AFTER,ei3,ei4,e3,PRESPART,shattering,0173 1999-03-13 Hungarian Leaders Hail Entry...,e4,NONE,World War II
1033,0,l40,BEFORE,ei24,ei25,e24,PAST,fled,"""Hallelujah,""(e23:exclaimed) Albright, a daug...",e25,PAST,became


Data Cleaning and fetching only AFTER and BEFORE relation

In [ ]:
data.Relation.value_counts()

BEFORE          469
AFTER           194
IDENTITY        124
INCLUDES         60
IS_INCLUDED      59
SIMULTANEOUS     41
IBEFORE          32
DURING           29
IAFTER           16
BEGINS            7
BEGUN_BY          4
Name: Relation, dtype: int64

In [ ]:
def relationcorection(relation):
    if relation=='IBEFORE':
        return 'BEFORE'
    elif  relation =='IAFTER':
        return 'AFTER'
    else:
        return relation

In [ ]:
data['Relation']=data['Relation'].apply(relationcorection)
data

,Index,lid,Relation,eiid1,eiid2,eventid1,e1tense,event1,context,eventid2,e2tense,event2
0,0,l3,BEFORE,ei1,ei2,e1,PAST,exploded,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e2,PRESPART,killing
1,1,l4,BEFORE,ei2,ei3,e2,PRESPART,killing,embassies in the Kenyan and Tanzanian capitals...,e3,PAST,said
2,2,l5,AFTER,ei3,ei12,e3,PAST,said,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e12,PAST,injured
3,3,l6,BEFORE,ei12,ei4,e12,PAST,injured,\t The American ambassador to Kenya was amon...,e4,PAST,said
4,4,l7,BEFORE,ei1,ei5,e1,PAST,exploded,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e5,PAST,said
...,...,...,...,...,...,...,...,...,...,...,...,...
1030,0,l30,AFTER,ei20,ei22,e20,PRESENT,finished,The three countries have (e20:finished) allne...,e22,PAST,ratified
1031,0,l7,BEFORE,ei5,ei6,e5,NONE,speeches,In (e5:speeches) before the signing (e6:ceremo...,e6,NONE,ceremony
1032,0,l5,AFTER,ei3,ei4,e3,PRESPART,shattering,0173 1999-03-13 Hungarian Leaders Hail Entry...,e4,NONE,World War II
1033,0,l40,BEFORE,ei24,ei25,e24,PAST,fled,"""Hallelujah,""(e23:exclaimed) Albright, a daug...",e25,PAST,became


In [ ]:
data.Relation.value_counts()

BEFORE          501
AFTER           210
IDENTITY        124
INCLUDES         60
IS_INCLUDED      59
SIMULTANEOUS     41
DURING           29
BEGINS            7
BEGUN_BY          4
Name: Relation, dtype: int64

In [ ]:
def verb_tense_check(e1tense,e2tense):
    if(e1tense == 'NONE') | (e2tense == 'NONE'):
        return 'NONE'
    elif e1tense == e2tense:
        return 'SAME'
    else:
        return 'DIFFERENT'


def count_events(sentence):
    pattern = r'\(e\d{1,3}:\w+\)'
    return len(re.findall(pattern, sentence))
def word_count(context):
    return len(context.split())

def sentence_count(context):
    return len(sent_tokenize(context))

In [ ]:
data['e1']=data['eventid1'].str[1:].astype(int)
data['e2']=data['eventid2'].str[1:].astype(int)
data['verb_tense']=data.apply(lambda x: verb_tense_check(x['e1tense'], x['e2tense']),
                        axis=1)
data['event1_pos']= np.where(data['e1']<data['e2'], 1, 2)
data.head(20)

,Index,lid,Relation,eiid1,eiid2,eventid1,e1tense,event1,context,eventid2,e2tense,event2,e1,e2,verb_tense,event1_pos
0,0,l3,BEFORE,ei1,ei2,e1,PAST,exploded,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e2,PRESPART,killing,1,2,DIFFERENT,1
1,1,l4,BEFORE,ei2,ei3,e2,PRESPART,killing,embassies in the Kenyan and Tanzanian capitals...,e3,PAST,said,2,3,DIFFERENT,1
2,2,l5,AFTER,ei3,ei12,e3,PAST,said,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e12,PAST,injured,3,12,SAME,1
3,3,l6,BEFORE,ei12,ei4,e12,PAST,injured,\t The American ambassador to Kenya was amon...,e4,PAST,said,12,4,SAME,2
4,4,l7,BEFORE,ei1,ei5,e1,PAST,exploded,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e5,PAST,said,1,5,SAME,1
5,5,l8,BEFORE,ei5,ei6,e5,PAST,said,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e6,PAST,refused,5,6,SAME,1
6,6,l10,BEFORE,ei9,ei10,e9,PAST,went,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e10,PAST,said,9,10,SAME,1
7,7,l11,BEFORE,ei1,ei8,e1,PAST,exploded,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e8,PRESENT,see,1,8,DIFFERENT,1
8,8,l12,AFTER,ei12,ei1,e12,PAST,injured,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e1,PAST,exploded,12,1,SAME,2
9,0,l5,INCLUDES,ei24,ei1,e24,NONE,bombings,APW19980808.0022 NEWS STORY 08/08/1998 00:5...,e1,PAST,blew,24,1,NONE,2


In [ ]:
data['no_of_events']= data['context'].apply(count_events)
data['word_count']= data['context'].apply(word_count)
data['sent_count']= data['context'].apply(sentence_count)
data.head(10)

,Index,lid,Relation,eiid1,eiid2,eventid1,e1tense,event1,context,eventid2,e2tense,event2,e1,e2,verb_tense,event1_pos,no_of_events,word_count,sent_count
0,0,l3,BEFORE,ei1,ei2,e1,PAST,exploded,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e2,PRESPART,killing,1,2,DIFFERENT,1,3,75,2
1,1,l4,BEFORE,ei2,ei3,e2,PRESPART,killing,embassies in the Kenyan and Tanzanian capitals...,e3,PAST,said,2,3,DIFFERENT,1,2,13,1
2,2,l5,AFTER,ei3,ei12,e3,PAST,said,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e12,PAST,injured,3,12,SAME,1,3,75,2
3,3,l6,BEFORE,ei12,ei4,e12,PAST,injured,\t The American ambassador to Kenya was amon...,e4,PAST,said,12,4,SAME,2,2,12,1
4,4,l7,BEFORE,ei1,ei5,e1,PAST,exploded,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e5,PAST,said,1,5,SAME,1,6,95,4
5,5,l8,BEFORE,ei5,ei6,e5,PAST,said,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e6,PAST,refused,5,6,SAME,1,8,103,4
6,6,l10,BEFORE,ei9,ei10,e9,PAST,went,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e10,PAST,said,9,10,SAME,1,11,126,5
7,7,l11,BEFORE,ei1,ei8,e1,PAST,exploded,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e8,PRESENT,see,1,8,DIFFERENT,1,11,126,5
8,8,l12,AFTER,ei12,ei1,e12,PAST,injured,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e1,PAST,exploded,12,1,SAME,2,5,87,3
9,0,l5,INCLUDES,ei24,ei1,e24,NONE,bombings,APW19980808.0022 NEWS STORY 08/08/1998 00:5...,e1,PAST,blew,24,1,NONE,2,1,79,2


In [ ]:
final_data=data[(data.Relation=='BEFORE') | (data.Relation=='AFTER')]
final_data

,Index,lid,Relation,eiid1,eiid2,eventid1,e1tense,event1,context,eventid2,e2tense,event2,e1,e2,verb_tense,event1_pos,no_of_events,word_count,sent_count
0,0,l3,BEFORE,ei1,ei2,e1,PAST,exploded,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e2,PRESPART,killing,1,2,DIFFERENT,1,3,75,2
1,1,l4,BEFORE,ei2,ei3,e2,PRESPART,killing,embassies in the Kenyan and Tanzanian capitals...,e3,PAST,said,2,3,DIFFERENT,1,2,13,1
2,2,l5,AFTER,ei3,ei12,e3,PAST,said,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e12,PAST,injured,3,12,SAME,1,3,75,2
3,3,l6,BEFORE,ei12,ei4,e12,PAST,injured,\t The American ambassador to Kenya was amon...,e4,PAST,said,12,4,SAME,2,2,12,1
4,4,l7,BEFORE,ei1,ei5,e1,PAST,exploded,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,e5,PAST,said,1,5,SAME,1,6,95,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,0,l30,AFTER,ei20,ei22,e20,PRESENT,finished,The three countries have (e20:finished) allne...,e22,PAST,ratified,20,22,DIFFERENT,1,3,18,1
1031,0,l7,BEFORE,ei5,ei6,e5,NONE,speeches,In (e5:speeches) before the signing (e6:ceremo...,e6,NONE,ceremony,5,6,NONE,1,4,25,1
1032,0,l5,AFTER,ei3,ei4,e3,PRESPART,shattering,0173 1999-03-13 Hungarian Leaders Hail Entry...,e4,NONE,World War II,3,4,NONE,1,4,47,1
1033,0,l40,BEFORE,ei24,ei25,e24,PAST,fled,"""Hallelujah,""(e23:exclaimed) Albright, a daug...",e25,PAST,became,24,25,SAME,1,3,18,1
